In [ ]:
# Programmatically create and query Tableau Agents using APIs

# Step 1: Run your agent using Langraph dev or Langgraph cloud

In [1]:
#import required packages
import requests
import json
from IPython.display import Markdown, display

In [9]:
#generate an instance of a Tableau Data Q&A Agent
def generate_agent(agent_url, graph_id_name):
    url = agent_url + "/assistants"
    
    payload = {
        "assistant_id": "",
        "graph_id": graph_id_name,
        "config": {},
        "metadata": {},
        "if_exists": "raise",
        "name": ""
    }
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(url, json=payload, headers=headers)
    
    return(response.json()["assistant_id"])

In [12]:
my_url = 'http://127.0.0.1:2024'
my_graph_id = 'keynote'
agent_id = generate_agent(my_url, my_graph_id)
agent_id

'17909892-e2f4-479b-825e-d97f4120b62e'

In [15]:

endpoint = "/runs/stream"
payload = {
    "assistant_id": agent_id,
    "input": {
        "messages": [
            ("human", "how much quantity do we purchase from each supplier?")
        ]
    }
}
response = requests.post(my_url+endpoint, json=payload)

In [17]:
def pretty_print_tab_agent_response(agent_output):
    current_event = None
    for line in agent_output.iter_lines():
        if not line:
            continue
    
        decoded = line.decode("utf-8")
    
        if decoded.startswith("event:"):
            current_event = decoded.replace("event: ", "").strip()
        elif decoded.startswith("data:"):
            data_json = decoded.replace("data: ", "").strip()
    
            try:
                data = json.loads(data_json)
            except json.JSONDecodeError:
                print(f"⚠️ Couldn't parse JSON: {data_json}")
                continue
    
            # 🎯 Final AI response usually in 'values' event
            if current_event == "values":
                messages = data.get("messages", [])
                for msg in messages:
                    if msg.get("type") == "ai" and msg.get("content"):
                        return(display(Markdown(msg["content"])))

pretty_print_tab_agent_response(response)

The data query has been successfully executed, and here are the total quantities purchased from each supplier:

| Supplier          | Total Quantity |
|-------------------|----------------|
| Easy Riders       | 48,639         |
| Vee Bike          | 192,102        |
| Road Runner       | 18,594         |
| Brews Brothers     | 65,048         |
| Power Bikes       | 48,427         |
| AllTech Group     | 31,519         |
| River Systems     | 75,822         |
| Greenholt Group   | 80,762         |
| Vivid Technologies | 43,216         |
| Roller Coaster    | 39,685         |

This data reflects the total quantities purchased from each supplier based on the available records. If you need further analysis or insights, please let me know!